In [1]:
import pandas as pd
import numpy as np
import math
import tensorflow as tf

2023-07-24 09:00:15.743508: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-24 09:00:15.924974: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-07-24 09:00:16.517459: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-07-24 09:00:16.517513: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [2]:
users_rating_df = pd.read_csv("ml-25m/ratings.csv")
movies_df = pd.read_csv("ml-25m/movies.csv")

In [3]:
users_rating_df.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [4]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
user_rating_movie_details_df = pd.merge(users_rating_df, movies_df, how="left", on="movieId")

In [6]:
user_rating_movie_details_df.head()

,userId,movieId,rating,timestamp,title,genres
0,1,296,5.0,1147880044,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
1,1,306,3.5,1147868817,Three Colors: Red (Trois couleurs: Rouge) (1994),Drama
2,1,307,5.0,1147868828,Three Colors: Blue (Trois couleurs: Bleu) (1993),Drama
3,1,665,5.0,1147878820,Underground (1995),Comedy|Drama|War
4,1,899,3.5,1147868510,Singin' in the Rain (1952),Comedy|Musical|Romance


Each user is garatee to have at least 20 reviews,
I have the time stamp thus I have a way to make a sequence.

In [7]:
# study for user 1

user_rating_movie_details_df.query("userId==1").sort_values(by="timestamp", ascending=True)

,userId,movieId,rating,timestamp,title,genres
36,1,5952,4.0,1147868053,"Lord of the Rings: The Two Towers, The (2002)",Adventure|Fantasy
13,1,2012,2.5,1147868068,Back to the Future Part III (1990),Adventure|Comedy|Sci-Fi|Western
12,1,2011,2.5,1147868079,Back to the Future Part II (1989),Adventure|Comedy|Sci-Fi
11,1,1653,4.0,1147868097,Gattaca (1997),Drama|Sci-Fi|Thriller
9,1,1250,4.0,1147868414,"Bridge on the River Kwai, The (1957)",Adventure|Drama|War
...,...,...,...,...,...,...
65,1,27193,3.0,1147879774,Taxi 2 (2000),Action|Comedy
32,1,5684,2.0,1147879797,Naqoyqatsi (2002),Documentary
45,1,7318,2.0,1147879850,"Passion of the Christ, The (2004)",Drama
0,1,296,5.0,1147880044,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller


In [8]:
# by movie

user_rating_movie_details_df.query("movieId==5952").sort_values(by=["userId", "timestamp"], ascending=True)

,userId,movieId,rating,timestamp,title,genres
36,1,5952,4.0,1147868053,"Lord of the Rings: The Two Towers, The (2002)",Adventure|Fantasy
212,2,5952,5.0,1141415528,"Lord of the Rings: The Two Towers, The (2002)",Adventure|Fantasy
453,3,5952,4.0,1439472616,"Lord of the Rings: The Two Towers, The (2002)",Adventure|Fantasy
972,4,5952,4.5,1574193472,"Lord of the Rings: The Two Towers, The (2002)",Adventure|Fantasy
2636,13,5952,4.5,1237970978,"Lord of the Rings: The Two Towers, The (2002)",Adventure|Fantasy
...,...,...,...,...,...,...
24998468,162532,5952,5.0,1418000735,"Lord of the Rings: The Two Towers, The (2002)",Adventure|Fantasy
24998762,162533,5952,5.0,1281406073,"Lord of the Rings: The Two Towers, The (2002)",Adventure|Fantasy
24999074,162534,5952,3.0,1527428422,"Lord of the Rings: The Two Towers, The (2002)",Adventure|Fantasy
24999701,162538,5952,4.0,1438781110,"Lord of the Rings: The Two Towers, The (2002)",Adventure|Fantasy


In [9]:
user_rating_movie_details_df[["userId", "movieId"]].groupby(by="userId").count().sort_values(by="movieId", ascending=False)

,movieId
userId,
72315,32202
80974,9178
137293,8913
33844,7919
20055,7488
...,...
12094,20
119539,20
156759,20


# Build a dataset
I need to build a rating matrix Y with a row per movie and column per each user's rating.

And another R matrix that will indicate if the user has given a rating to that moview(1) or not(0)

For that to avoid having a super big matrix I will chose the first 10000 movies first

In [288]:
# get all movies ids
all_movies_ids = users_rating_df.movieId.unique()

# choose the first 5000
sampled_movies_ids = all_movies_ids[:5000]
# keep all user's rating for the fist 5000 movies
rating_from_sampled_movies_df = users_rating_df[users_rating_df.movieId.isin(sampled_movies_ids)]

#First 500 users
sampled_users_id = rating_from_sampled_movies_df.userId.unique()[:500]
rating_from_sampled_movies_df = rating_from_sampled_movies_df[users_rating_df.userId.isin(sampled_users_id)]

/tmp/ipykernel_3735/2651256587.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  rating_from_sampled_movies_df = rating_from_sampled_movies_df[users_rating_df.userId.isin(sampled_users_id)]


In [11]:
Nu = sampled_users_id.shape[0]
Nm = sampled_movies_ids.shape[0]

In [12]:
print(f"Users: {Nu} - Movies:{Nm}")

Users: 500 - Movies:5000


In [13]:
# All users rating per movie.
Y = np.zeros((Nm, Nu), dtype="float16")

# Mask 1 if user rated that movie 0 if not.
R = np.zeros((Nm, Nu))

#Build first the matrices

2 dictionaries

The first one will have a all the users and the second one the movies

Those dictionaries will help me to kee track of which movie is the one that is in each row of the matrix and which use is the one in each column

In [14]:
movies_index_dict = dict(zip(rating_from_sampled_movies_df.movieId.unique(), range(rating_from_sampled_movies_df.movieId.unique().shape[0])))
users_index_dict = dict(zip(rating_from_sampled_movies_df.userId.unique(), range(rating_from_sampled_movies_df.userId.unique().shape[0])))

Test with a user that haven seen the movie and another that did see the movie

In [15]:
movie_id = 300
user_id = 0
rating = rating_from_sampled_movies_df.query(f"movieId=={movie_id} and userId=={user_id}").rating.to_numpy()
assert rating.shape[0] == 0

user_have_seen_movie = rating_from_sampled_movies_df.query(f"movieId=={movie_id}").userId.unique()[0]
rating = rating_from_sampled_movies_df.query(f"movieId=={movie_id} and userId=={user_have_seen_movie}").rating.to_numpy()
assert rating.shape[0] > 0

In [16]:
from tqdm import tqdm

In [17]:
for movie_id in tqdm(sampled_movies_ids, total=sampled_movies_ids.shape[0]):
    for user_id in rating_from_sampled_movies_df.query(f"movieId=={movie_id}").userId.to_numpy():
        rating = rating_from_sampled_movies_df.query(f"movieId=={movie_id} and userId=={user_id}").rating.to_numpy()
        Y[movies_index_dict[movie_id], users_index_dict[user_id]] = rating[0]
        R[movies_index_dict[movie_id], users_index_dict[user_id]] = 1

100%|██████████| 5000/5000 [01:02<00:00, 80.07it/s] 


# Split train, val and test set

In [18]:
Y[0, :10]

array([5. , 0. , 5. , 4. , 4. , 0. , 4. , 5. , 0. , 4.5], dtype=float16)

In [19]:
# 70% for training
N = Y.shape[0]

n_train = math.ceil(N*0.7)
n_val = math.ceil(N*0.15)
n_test = N-n_train-n_val

np.random.RandomState(42).shuffle(Y)
np.random.RandomState(42).shuffle(R)

Y_train = Y[:n_train]
Y_val = Y[n_train:n_train+n_val]
Y_test = Y[n_train+n_val:]

print(f"Train N: {Y_train.shape}, Val N:{Y_val.shape}, Test:{Y_test.shape}")

Train N: (3500, 500), Val N:(750, 500), Test:(750, 500)


# Colaborative filtering

# normalize values rating per movie

In [20]:
first_movie_mean = np.mean(Y[0, R[0, :].astype(bool)])

In [21]:
Y[Y == 0] = np.nan
means = np.nanmean(Y, axis=1)
means = means.reshape(Nm, 1)

In [22]:
Y -= means

In [23]:
assert means[0] == first_movie_mean

In [24]:
# revert nan conversion
Y = np.nan_to_num(Y)

# algorithm!

# Try with an without linalg and with and without diving by Nu*Nm

In [80]:
# learn parameters W X and B
# W users
# X movies features to learn

# number of features for users and movies?
Nf = 100
learning_rate = 1e-1
lambda_reg = 1.5

W = tf.Variable(np.random.ranf((Nu, Nf)), dtype='float32')
X = tf.Variable(np.random.ranf((Nm, Nf)), dtype='float32')
b = tf.Variable(np.random.ranf((1, Nu)), dtype="float32")
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

for epoch in tqdm(range(500)):
    with tf.GradientTape() as tape:
        # pred_rating = tf.linalg.matmul(X, tf.transpose(W))+b
        pred_rating = tf.matmul(X, tf.transpose(W))+b
        # Add normalization terms for W and X L2 norm
        loss = tf.reduce_sum(((pred_rating-Y)*R)**2)
        l2_reg = (lambda_reg/2)*((tf.reduce_sum(W**2))+(tf.reduce_sum(X**2)))
        loss += l2_reg
        loss/=(Nm*Nu)

    gradients = tape.gradient(loss, [W, X, b])
    optimizer.apply_gradients(zip(gradients, [W, X, b]))

    if epoch%25==0:
        print(f"loss: {loss}")

  3%|▎         | 14/500 [00:00<00:07, 65.59it/s]

loss: 15.42919635772705


  7%|▋         | 35/500 [00:00<00:06, 67.24it/s]

loss: 0.06498824805021286


 13%|█▎        | 63/500 [00:00<00:06, 66.79it/s]

...

 92%|█████████▏| 462/500 [00:07<00:00, 65.75it/s]

loss: 0.002874556928873062


 97%|█████████▋| 483/500 [00:07<00:00, 65.05it/s]

loss: 0.0027865266893059015


100%|██████████| 500/500 [00:07<00:00, 65.40it/s]


In [81]:
# index to user id
users_reversed_index_dict = dict([(index, id) for id, index in users_index_dict.items()])

# reversed dict movies
movies_reversed_index_dict = dict([(index, id) for id, index in movies_index_dict.items()])

In [82]:
# Make predictions for all user's ranting
ranking_preds = (tf.matmul(X, tf.transpose(W))+b) + means

In [96]:
# for user 0
user_index_position = 100

print(f"UserId : {users_reversed_index_dict[0]} in index position: {user_index_position}")

user_ranking_predictions = ranking_preds[:, user_id]
user_rated_movies_mask = R[:, user_id]
user_rating = (Y + means)[:, user_id]

for movie_index, user_rating_mask in enumerate(user_rated_movies_mask):
    movie_title = movies_df.query(f"movieId=={movies_reversed_index_dict[movie_index]}").title.to_numpy()[0]
    user_predicted_ranking = user_ranking_predictions[movie_index]
    if user_rating_mask:
        print(f"Movie: {movie_title} | user rating: {user_rating[movie_index]} | predicted rating:{user_predicted_ranking}")
    else:
        if user_predicted_ranking>=5:
            print(f"Recommended movie that is likely to rank over 4: {movie_title} | predicted ranking: {user_predicted_ranking}")

UserId : 1 in index position: 100
Movie: Barbarian Invasions, The (Les invasions barbares) (2003) | user rating: 4.5 | predicted rating:4.498409271240234
Movie: M. Hulot’s Holiday (Mr. Hulot's Holiday) (Vacances de Monsieur Hulot, Les) (1953) | user rating: 3.0 | predicted rating:2.969052791595459
Movie: Bad Education (La mala educación) (2004) | user rating: 5.0 | predicted rating:4.947770118713379
Recommended movie that is likely to rank over 4: Sound of Music, The (1965) | predicted ranking: 6.217047214508057
Movie: Romancing the Stone (1984) | user rating: 5.0 | predicted rating:4.905855655670166
Recommended movie that is likely to rank over 4: Muppet Movie, The (1979) | predicted ranking: 5.059998989105225
Movie: X2: X-Men United (2003) | user rating: 5.0 | predicted rating:4.952508926391602
Movie: Apocalypse Now (1979) | user rating: 2.5 | predicted rating:2.4893298149108887
Recommended movie that is likely to rank over 4: Fast and the Furious, The (2001) | predicted ranking: 5.4

# TODO:

# 1- find similar movies based on cosine distance

# 2- Limit recomendations up to top N
# 3- Add a new user np.c_ to concatenate and add a new users rating

## Strategy: Decide which user and movies to select
##  I would choose the latest 5 years movies pluss iconic movies
## users with more than 20 ratings, active users
## If I want to release a new funcionality
## I would select past active users from last month who paid for premium to bring them back before we launch the new thing

In [104]:
def cosine_distance(movie_index_1, movie_index_2):
    return tf.matmul(tf.expand_dims(W[movie_index_1, :], axis=0), tf.transpose(tf.expand_dims(W[movie_index_2, :], axis=0)))/(tf.reduce_sum(W[movie_index_1, :]**2)+tf.reduce_sum(W[movie_index_2, :]**2))


In [105]:
cosine_distance(0, 0)

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.50000006]], dtype=float32)>